In [110]:
from sentence_transformers import SentenceTransformer
from pandarallel import pandarallel
model = SentenceTransformer('all-mpnet-base-v2')

In [111]:
# print(sample_encode1.shape, sample_encode2.shape)

In [112]:
# local_model = SentenceTransformer('/home/linhnm/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2')

In [113]:
# vec = local_model.encode('hello London')
# print(vec)

In [114]:
base_path = '../data'
raw_path = f'{base_path}/raw'
vectorize_path = f'{base_path}/vectorize'

In [115]:
import pandas as pd
import numpy as np
import json

In [116]:
dataset = 'tmp'
# dataset = 'Movies_and_TV'
batch_size = 1000

In [117]:
def save_csv(overall, review_text, batch_idx):
  df = pd.DataFrame({
    'review_text': review_text,
    'overall': overall
  })
  
  df['review_text'].to_csv(f'{raw_path}/{dataset}/X_{batch_idx}.csv', index=False)
  df['overall'].to_csv(f'{raw_path}/{dataset}/y_{batch_idx}.csv', index=False)

In [118]:

cols = ['overall', 'reviewText']
overall = []
review_text = []

file_name = f'{raw_path}/{dataset}/{dataset}.json'

i = 0
batch_idx = 0

with open(file_name, encoding='utf8') as f:
    for line in f:
        i = i + 1
        
        doc = json.loads(line)
        overall.append(doc.get('overall'))
        review_text.append(doc.get('reviewText'))
          
        if i % batch_size == 0 :
          batch_idx = i//batch_size - 1
          save_csv(overall, review_text, batch_idx)
          overall = []
          review_text = []
        
batch_idx = batch_idx + 1
save_csv(overall, review_text, batch_idx)
num_batch = batch_idx + 1

In [119]:
def validate(batch_num):
  nalist = []
  for i in range(batch_num):
    X = pd.read_csv(f'{base_path}/raw/{dataset}/X_{i}.csv')
    y = pd.read_csv(f'{base_path}/raw/{dataset}/y_{i}.csv')
    
    tona_X = pd.isnull(X).sum().item()
    tona_y = pd.isnull(y).sum().item()
    
    if (tona_X > 0) or (tona_y > 0):
      print(f'Dirty data for batch: {i}, nan X: {tona_X}, nan y: {tona_y}')
      nalist.append(i)
  return nalist

In [120]:
confuse = []
for i in range(num_batch):
  X = pd.read_csv(f'{base_path}/raw/{dataset}/X_{i}.csv')
  y = pd.read_csv(f'{base_path}/raw/{dataset}/y_{i}.csv')
  
  tona_X = pd.isnull(X).sum().item()
  tona_y = pd.isnull(y).sum().item()
  
  if (tona_X > 0) or (tona_y > 0):
    print(f'Dirty data for batch: {i}, nan X: {tona_X}, nan y: {tona_y}')
    tmp = X.isna().any(axis=1)
    naidx = tmp[tmp].index.tolist()
    X = X.drop(naidx)
    y = y.drop(naidx)
    
    sx = X.shape[0]
    sy = y.shape[0]
    
    if sx != sy:
      print(f'confuse, batch: {i}, sx: {sx}, sy: {sy}')
      confuse.append({
        'batch': i,
        'sx': sx,
        'sy': sy
      })

  X.to_csv(f'{base_path}/raw/{dataset}/X_{i}.csv', index=False)
  y.to_csv(f'{base_path}/raw/{dataset}/y_{i}.csv', index=False)
  
    
nalist = validate(num_batch)
print(f'nalist after clean: {nalist}')
print(confuse)

Dirty data for batch: 3, nan X: 1, nan y: 0
Dirty data for batch: 5, nan X: 1, nan y: 0
Dirty data for batch: 6, nan X: 1, nan y: 0
Dirty data for batch: 7, nan X: 1, nan y: 0
Dirty data for batch: 9, nan X: 1, nan y: 0
Dirty data for batch: 11, nan X: 1, nan y: 0
Dirty data for batch: 12, nan X: 2, nan y: 0
Dirty data for batch: 13, nan X: 1, nan y: 0
Dirty data for batch: 14, nan X: 1, nan y: 0
Dirty data for batch: 17, nan X: 7, nan y: 0
Dirty data for batch: 18, nan X: 4, nan y: 0
Dirty data for batch: 19, nan X: 5, nan y: 0
Dirty data for batch: 20, nan X: 1, nan y: 0
Dirty data for batch: 21, nan X: 2, nan y: 0
Dirty data for batch: 22, nan X: 1, nan y: 0
Dirty data for batch: 23, nan X: 1, nan y: 0
Dirty data for batch: 24, nan X: 2, nan y: 0
Dirty data for batch: 26, nan X: 1, nan y: 0
Dirty data for batch: 27, nan X: 2, nan y: 0
Dirty data for batch: 28, nan X: 1, nan y: 0
Dirty data for batch: 31, nan X: 1, nan y: 0
Dirty data for batch: 33, nan X: 2, nan y: 0
Dirty data for 

In [121]:
for i in range(num_batch):
  X = pd.read_csv(f'{base_path}/raw/{dataset}/X_{i}.csv')
  y = pd.read_csv(f'{base_path}/raw/{dataset}/y_{i}.csv')
  
  sx = X.shape[0]
  sy = y.shape[0]
  
  if (sx != sy):
    print(f'confuse, batch: {i}, sx: {sx}, sy: {sy}')

confuse, batch: 352, sx: 999, sy: 1000
confuse, batch: 804, sx: 998, sy: 999
confuse, batch: 1267, sx: 998, sy: 999
confuse, batch: 1548, sx: 996, sy: 997
confuse, batch: 1711, sx: 999, sy: 1000
confuse, batch: 4574, sx: 996, sy: 997
confuse, batch: 4984, sx: 999, sy: 1000
confuse, batch: 5214, sx: 998, sy: 999
confuse, batch: 6140, sx: 998, sy: 999
confuse, batch: 6345, sx: 997, sy: 998
confuse, batch: 6414, sx: 998, sy: 999
confuse, batch: 6604, sx: 997, sy: 998
confuse, batch: 6979, sx: 995, sy: 996
confuse, batch: 7137, sx: 988, sy: 989
confuse, batch: 7706, sx: 998, sy: 999
confuse, batch: 7778, sx: 998, sy: 999
confuse, batch: 8156, sx: 999, sy: 1000
confuse, batch: 8278, sx: 998, sy: 999
confuse, batch: 8581, sx: 998, sy: 999


In [122]:
print(X.info())
print(y.info())

print(X.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review_text  568 non-null    object
dtypes: object(1)
memory usage: 4.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   overall  568 non-null    float64
dtypes: float64(1)
memory usage: 4.6 KB
None
(568, 1)


In [123]:
pd.isnull(X).sum()

review_text    0
dtype: int64

In [124]:
# X_vec = X.apply(lambda row : model.encode(row)[0], axis=1)

In [125]:
# type(X_vec.iloc[0])
# X_vec.iloc[0].shape

# X_vec.head()

In [126]:
# Xnp = np.array(X_vec.values.tolist())

In [127]:
# Xnp.shape

In [128]:
# Xnp[0]